In [1]:
import sys

from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

In [2]:
weights = MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = mobilenet_v3_large(weights=weights)

print(model)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [3]:
from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
model = SiameseDuplicateImageNetwork().to(device)

In [6]:
train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [7]:
feature1, feature2, _ = next(iter(train_dataloader))
feature1, feature2

(tensor([[[[248, 248, 248,  ..., 233, 235, 235],
           [248, 248, 248,  ..., 233, 235, 235],
           [249, 249, 249,  ..., 233, 235, 234],
           ...,
           [159, 160, 149,  ..., 232, 238, 236],
           [180, 180, 178,  ..., 221, 233, 238],
           [186, 182, 177,  ..., 220, 222, 231]],
 
          [[225, 225, 225,  ..., 227, 226, 226],
           [225, 225, 225,  ..., 227, 226, 226],
           [226, 226, 226,  ..., 227, 226, 225],
           ...,
           [109, 109, 102,  ..., 224, 230, 226],
           [129, 125, 121,  ..., 208, 224, 230],
           [132, 130, 122,  ..., 208, 213, 226]],
 
          [[194, 194, 194,  ..., 221, 221, 221],
           [194, 194, 194,  ..., 221, 221, 221],
           [195, 195, 195,  ..., 221, 221, 220],
           ...,
           [ 64,  64,  58,  ..., 216, 226, 220],
           [ 79,  72,  65,  ..., 199, 220, 227],
           [ 84,  82,  72,  ..., 195, 204, 220]]],
 
 
         [[[178, 175, 173,  ..., 163, 164, 163],
         

In [8]:
feature1 = feature1.to(device)
feature2 = feature2.to(device)

output = model(feature1, feature2)
print(output)

C:\Users\ruiqinng\miniconda3\envs\dai\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor([[-0.0799],
        [-0.0835],
        [-0.0808],
        [-0.0787],
        [-0.0797],
        [-0.0795],
        [-0.0850],
        [-0.0853],
        [-0.0802],
        [-0.0801],
        [-0.0811],
        [-0.0830],
        [-0.0877],
        [-0.0794],
        [-0.0814],
        [-0.0812],
        [-0.0800],
        [-0.0827],
        [-0.0819],
        [-0.0839],
        [-0.0856],
        [-0.0875],
        [-0.0790],
        [-0.0819],
        [-0.0807],
        [-0.0756],
        [-0.0847],
        [-0.0822],
        [-0.0845],
        [-0.0842],
        [-0.0822],
        [-0.0803],
        [-0.0851],
        [-0.0775],
        [-0.0840],
        [-0.0815],
        [-0.0800],
        [-0.0838],
        [-0.0816],
        [-0.0764],
        [-0.0832],
        [-0.0854],
        [-0.0812],
        [-0.0790],
        [-0.0786],
        [-0.0852],
        [-0.0839],
        [-0.0818],
        [-0.0802],
        [-0.0864],
        [-0.0790],
        [-0.0803],
        [-0.

In [10]:
output.shape

torch.Size([64, 1])